<a href="https://colab.research.google.com/github/kurtsenol/machine-learning/blob/master/TopicModeling/Methodology/prodigy_preprocess_220205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

import re
import random
import unicodedata
import string
import json

In [ ]:
!pip install Pattern
from pattern.text.en import singularize

In [ ]:
#spaCy
import spacy

#nlp = spacy.load('en_core_web_md')
nlp = spacy.load('en_core_web_sm')

# Import the Matcher
from spacy.matcher import Matcher

In [ ]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [ ]:
def clean(s):
  printable = set(string.printable)
  return ''.join(filter(lambda x: x in printable, s))

In [ ]:
clean("Research on sustainability in tourism has incr..")

'Research on sustainability in tourism has incr..'

In [ ]:
strip_chars = string.punctuation + " "
strip_chars

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

In [ ]:
df_taxonomy = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/entity_list_06.12.xlsx")
df_taxonomy.head(60)

,Unnamed: 0,Entity_List
0,0,TELEPHONE SURVEY
1,1,DISCOURSE
2,2,TIME-SERIES CROSS-SECTION
3,3,SELF ORGANIZING MAPS
4,4,QUOTA SAMPLING
5,5,SCATTER PLOT
6,6,TREND ANALYSIS
7,7,DBNS
8,8,POINTINTERP
9,9,NARRATIVE INTERVIEWING


In [ ]:
df_taxonomy[df_taxonomy["Entity_List"].str.contains("Principal Component")]

,Unnamed: 0,Entity_List


In [ ]:
df_taxonomy["Entity_List"] = df_taxonomy["Entity_List"].str.strip()
df_taxonomy["Entity_List"] = df_taxonomy["Entity_List"].str.replace("ANAYLISIS", "ANALYSIS")
df_taxonomy = df_taxonomy[df_taxonomy["Entity_List"]!="CONTINUOUS"]

In [ ]:
tax_list = df_taxonomy["Entity_List"].unique().tolist()

In [ ]:
# remove "\xa0" from strings and uppercase 
tax_list = [unicodedata.normalize("NFKD",t).upper() for t in tax_list]
tax_list = [t.strip("\u200e") for t in tax_list]

tax_list = [t.upper() if len(t)<6 else t.lower() for t in tax_list]
tax_list = list(set(tax_list))
tax_list

['social indicator',
 'systematic sampling',
 'NVIVO',
 'dot distribution map',
 'PLS',
 'qualitative comparative analysis',
 'telephone survey',
 'DBNS',
 'statistical packages',
 'theoretical sampling',
 'lemmatization',
 'historical research',
 'clustering',
 'cross-national research',
 'heterogeneity sampling',
 'observation',
 'semantic role labelling',
 'online research',
 'granger causality',
 'crucial experiment',
 'diffusion of treatment',
 'triangulation',
 'internal validity',
 'laboratory experiment',
 'interpretive phenomenological analysis',
 'control group',
 'accidental sampling',
 'dictionary learning',
 'controlled observation',
 'gaussian mixture',
 'panel data',
 'simple random sampling',
 'target population',
 'expert sampling',
 'deductive',
 'survival analysis',
 'multidimensional scaling',
 'ECM',
 'CTA',
 'text and speech processing',
 'orthogonal design',
 'grounded theory',
 'RBFNS',
 'latitude',
 'conceptual analysis',
 'cybermetrics',
 'scenario experiment'

In [ ]:
df_tax_org = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/Taxonomy list.xlsx", usecols="A:D")
for col in df_tax_org:
  df_tax_org[col] = df_tax_org[col].str.strip()
df_tax_org

,1st,2nd,3rd,4th
0,Qualitative data analysis,Coding,Coding schema,NaN
1,Qualitative data analysis,Coding,In-vivo coding,NaN
2,Qualitative data analysis,Coding,Memoing,NaN
3,Qualitative data analysis,Grounded theory,Open coding,NaN
4,Qualitative data analysis,Grounded theory,Axial coding,NaN
...,...,...,...,...
562,Implicit Association Test,NaN,NaN,NaN
563,IAT,NaN,NaN,NaN
564,Analytical Research,NaN,NaN,NaN
565,Fundamental Research,NaN,NaN,NaN


In [ ]:
df_tax_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1st     567 non-null    object
 1   2nd     538 non-null    object
 2   3rd     417 non-null    object
 3   4th     163 non-null    object
dtypes: object(4)
memory usage: 17.8+ KB


In [ ]:
tax_org_list = sorted(df_tax_org["1st"].dropna().unique().tolist() + df_tax_org["2nd"].dropna().unique().tolist() + df_tax_org["3rd"].dropna().unique().tolist() + df_tax_org["4th"].dropna().unique().tolist())
tax_org_list = [t.replace("\u200e", " ") for t in tax_org_list]
tax_org_list = [t.replace("\xa0", " ") for t in tax_org_list]
tax_org_list = [t.strip() for t in tax_org_list]
tax_org_list = [t if len(t)<6 else t.lower() for t in tax_org_list]
tax_org_list = [t.upper() if len(t)<4 else t for t in tax_org_list]

tax_org_list = sorted(list(set(tax_org_list)))
tax_org_list

['',
 'ANOVA',
 'ARIMA',
 'ARL',
 'BIRCH',
 'BNIM',
 'BOW',
 'CA',
 'CAPI',
 'CCA',
 'CDA',
 'CFA',
 'CN2',
 'CNNs',
 'CRA',
 'CTA',
 'CTM',
 'CorEx',
 'DBNs',
 'DCM',
 'DRL',
 'ECA',
 'ECM',
 'EEG',
 'EFA',
 'GANs',
 'GBT',
 'GDA',
 'GLM',
 'GMM',
 'GPC',
 'GPR',
 'GSR',
 'HLLE',
 'HMM',
 'IAT',
 'ICA',
 'IDW',
 'IPA',
 'IRL',
 'ITS',
 'KDE',
 'KNN',
 'KRR',
 'LCA',
 'LDA',
 'LLE',
 'LSA',
 'LSTMs',
 'LTSA',
 'MCA',
 'MDS',
 'MLPs',
 'MLR',
 'NER',
 'NLP',
 'NMF',
 'NNIDW',
 'NVivo',
 'OCR',
 'OLS',
 'PAM',
 'PCA',
 'QDA',
 'RBFNs',
 'RBMs',
 'RNNs',
 'RTA',
 'Rulex',
 'SEM',
 'SGD',
 'SLR',
 'SOMs',
 'SPSS',
 'SRL',
 'STM',
 'SVM',
 'Stata',
 'TSCS',
 'TTS',
 'UMAP',
 'WSD',
 'abduction reasoning',
 'accidental sampling',
 'action research',
 'adaboost',
 'adaptive boosting',
 'adaptive conjoint analysis',
 'affinity propagation',
 'agglomerative clustering',
 'altmetrics',
 'analysis of covariance',
 'analysis of variance',
 'analytical research',
 'ancova',
 'anomaly detection',
 '

In [ ]:
l_index = tax_org_list.index('abduction reasoning')
l_index

82

In [ ]:
tax_org_list = [t.rstrip("s") if tax_org_list.index(t)< l_index else t for t in tax_org_list]

In [ ]:
plurals = [t for t in tax_org_list if singularize(t)!=t in tax_org_list]
plurals

['CAPI',
 'CTA',
 'ITS',
 'MDS',
 'OLS',
 'RTA',
 'SPSS',
 'Stata',
 'TSCS',
 'TTS',
 'adaptive conjoint analysis',
 'altmetrics',
 'asymmetric measures',
 'atlas.ti',
 'autoencoders',
 'bag of words',
 'balanced iterative reducing and clustering using hierarchies',
 'bayesian statistics',
 'bibliographic analysis',
 'bibliometrics',
 'brain signals',
 'canonical correlation analysis',
 'caqdas',
 'choice-based conjoint analysis',
 'citation analysis',
 'co-author analysis',
 'co-citation analysis',
 'co-word analysis',
 'cohort analysis',
 'cohort studies',
 'computer assisted qualitative data analysis',
 'conceptual analysis',
 'confidence intervals',
 'confirmatory factor analysis',
 'conjoint analysis',
 'content analysis',
 'continuous',
 'controlled observations',
 'conversation analysis',
 'convolutional neural networks',
 'correspondence analysis',
 'critical discourse analysis',
 'critical resistance analysis',
 'cross tabulations',
 'cross-case analysis',
 'cross-sectional su

In [ ]:
dups = [t for t in plurals if singularize(t) in tax_org_list] 
dups

['autoencoders',
 'k-nearest neighbors',
 'latent variable models',
 'neural networks',
 'scatter plots',
 'telephone surveys']

In [ ]:
for t in dups + [""]:
  tax_org_list.remove(t)

In [ ]:
tax_org_list

['ANOVA',
 'ARIMA',
 'ARL',
 'BIRCH',
 'BNIM',
 'BOW',
 'CA',
 'CAPI',
 'CCA',
 'CDA',
 'CFA',
 'CN2',
 'CNN',
 'CRA',
 'CTA',
 'CTM',
 'CorEx',
 'DBN',
 'DCM',
 'DRL',
 'ECA',
 'ECM',
 'EEG',
 'EFA',
 'GAN',
 'GBT',
 'GDA',
 'GLM',
 'GMM',
 'GPC',
 'GPR',
 'GSR',
 'HLLE',
 'HMM',
 'IAT',
 'ICA',
 'IDW',
 'IPA',
 'IRL',
 'ITS',
 'KDE',
 'KNN',
 'KRR',
 'LCA',
 'LDA',
 'LLE',
 'LSA',
 'LSTM',
 'LTSA',
 'MCA',
 'MDS',
 'MLP',
 'MLR',
 'NER',
 'NLP',
 'NMF',
 'NNIDW',
 'NVivo',
 'OCR',
 'OLS',
 'PAM',
 'PCA',
 'QDA',
 'RBFN',
 'RBM',
 'RNN',
 'RTA',
 'Rulex',
 'SEM',
 'SGD',
 'SLR',
 'SOM',
 'SPSS',
 'SRL',
 'STM',
 'SVM',
 'Stata',
 'TSCS',
 'TTS',
 'UMAP',
 'WSD',
 'abduction reasoning',
 'accidental sampling',
 'action research',
 'adaboost',
 'adaptive boosting',
 'adaptive conjoint analysis',
 'affinity propagation',
 'agglomerative clustering',
 'altmetrics',
 'analysis of covariance',
 'analysis of variance',
 'analytical research',
 'ancova',
 'anomaly detection',
 'applied resear

In [ ]:
df_feedback = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/merged_feedback.xlsx", header = None)
df_feedback[0] = df_feedback[0].str.strip()
df_feedback

,0
0,AHP
1,AMOS
2,Analytic Hierarchy Process
3,Analytical Hiearachy Process
4,Analytical Network Process
...,...
189,Unstructured Interview
190,Value-Focused Thinking
191,Variance analysis
192,VECM


In [ ]:
tax_feedback_list = sorted(df_feedback[0].dropna().unique().tolist())
tax_feedback_list = [t if len(t)<6 else t.lower() for t in tax_feedback_list]
tax_feedback_list = [t.upper() if len(t)<4 else t for t in tax_feedback_list]

tax_feedback_list = sorted(list(set(tax_feedback_list)))
tax_feedback_list

['AHP',
 'AMOS',
 'ANN',
 'ANOVA',
 'ANP',
 'ARDL',
 'BERT',
 'CDM',
 'DARE',
 'FANP',
 'GIS',
 'GRU',
 'HE',
 'MAE',
 'MCDA',
 'MLR',
 'MORE',
 'MORG',
 'MSE',
 'NGT',
 'PCA',
 'PPIGS',
 'PSO',
 'Panel',
 'SEM',
 'SODA',
 'SPSS',
 'SQISR',
 'SWOT',
 'TDC',
 'VECM',
 'analytic hierarchy process',
 'analytical hiearachy process',
 'analytical network process',
 'applied research',
 'bibliometric',
 'bioclimatic method',
 'caption evaluation method',
 'cartographic method',
 'case study',
 'casual research',
 'chi-square independent test',
 'chisquare analysis',
 'cluster analysis',
 'cluster sampling',
 'comparative analysis',
 'comparative approach',
 'complex social system',
 'complexity theory',
 'comprehensive benefit evaluation model',
 'concor',
 'conducting centrality analysis',
 'configurationist method',
 'confirmatory factor analysis',
 'construct equation analysis',
 'content analysis',
 'context analysis',
 'conventional design method',
 'conventional method',
 'copula model

In [ ]:
out_final = ["CNNS", "GANS", "LSTMS", "MLPS", "RBFNS", "RBMS", "RNNS", 'RULEX', 'SOMS', 'STATA', 'autoencoders', 'bibliometrics', 
             'box-and-whisker plot', 'brain signals', 'continuous', 'critical resistance analysis', 'decision trees', 'discourse',
             'discrete', 'ensemble methods',  'focus groups', 'fram analysis', 'hidden markov mode', 'hierarchal clustering', 'iSRL',
             'latent variable model', 'meta analysis', 'naturalistic observations', 'neural networks', 'nominal', 'non negative matrix factorization',
             'online surveys', 'participant observations', 'secondary data analysis', 'structural equation modeling', 'structural equation modeling technique',
             'support vector machines', 'true experimental research design', 'universe', 'controlled observations']
             


In [ ]:
tax_final_list = sorted(list(set(tax_list + tax_org_list + tax_feedback_list)))
print(len(tax_final_list))
tax_final_list

965


['AHP',
 'AMOS',
 'ANN',
 'ANOVA',
 'ANP',
 'ARDL',
 'ARIMA',
 'ARL',
 'AVE',
 'BERT',
 'BIRCH',
 'BNIM',
 'BOW',
 'CA',
 'CAPI',
 'CASAI',
 'CATI',
 'CCA',
 'CDA',
 'CDM',
 'CFA',
 'CN2',
 'CNN',
 'CNNS',
 'COREX',
 'CRA',
 'CTA',
 'CTM',
 'CorEx',
 'DARE',
 'DBN',
 'DBNS',
 'DCM',
 'DRL',
 'ECA',
 'ECM',
 'EDA',
 'EEG',
 'EFA',
 'FANP',
 'GAN',
 'GANS',
 'GBT',
 'GDA',
 'GIS',
 'GLM',
 'GMM',
 'GPC',
 'GPR',
 'GRU',
 'GSR',
 'HE',
 'HLLE',
 'HMM',
 'IAT',
 'ICA',
 'IDW',
 'IPA',
 'IRL',
 'ISRL',
 'ITS',
 'JASP',
 'KDE',
 'KNN',
 'KRR',
 'LCA',
 'LDA',
 'LLE',
 'LSA',
 'LSTM',
 'LSTMS',
 'LTSA',
 'MAE',
 'MCA',
 'MCDA',
 'MDS',
 'MLP',
 'MLPS',
 'MLR',
 'MORE',
 'MORG',
 'MSE',
 'NER',
 'NGT',
 'NLP',
 'NMF',
 'NNIDW',
 'NVIVO',
 'NVivo',
 'OCR',
 'OLS',
 'PAM',
 'PANEL',
 'PCA',
 'PLS',
 'PLSA',
 'PPIGS',
 'PROBE',
 'PSO',
 'Panel',
 'QDA',
 'RBFN',
 'RBFNS',
 'RBM',
 'RBMS',
 'RNN',
 'RNNS',
 'RTA',
 'RULEX',
 'Rulex',
 'SEM',
 'SGD',
 'SLR',
 'SNA',
 'SODA',
 'SOM',
 'SOMS',
 'SPSS

In [ ]:
for t in out_final:
  tax_final_list.remove(t)
print(len(tax_final_list))

925


In [ ]:
tax_final_list[-5:]

['word cloud',
 'word counting',
 'word sense disambiguation',
 'wrapper method',
 'z-test']

In [ ]:
for tax in tax_final_list[-5:]:
  doc = nlp(tax)
  for token in doc:
    # print(token)
    print(token.text)
    print(token.lemma_)
    # print("-" in token.lemma_)

    # print(type(token))
    # print(type(token.text))
    # print(type(token.lemma_))


word
word
cloud
cloud
word
word
counting
counting
word
word
sense
sense
disambiguation
disambiguation
wrapper
wrapper
method
method
z
z
-
-
test
test


In [ ]:
phrases = [t for t in tax_final_list if len(t)>5]
phrases[:5]

['abduction',
 'abduction reasoning',
 'abductive',
 'accidental sampling',
 'action research']

In [ ]:
phr_patterns = [[{"LOWER": t.text} if t.text != "-" else {"ORTH": "-", "OP": "?"} for t in nlp(word)] for word in phrases]
phr_patterns[:5]

[[{'LOWER': 'abduction'}],
 [{'LOWER': 'abduction'}, {'LOWER': 'reasoning'}],
 [{'LOWER': 'abductive'}],
 [{'LOWER': 'accidental'}, {'LOWER': 'sampling'}],
 [{'LOWER': 'action'}, {'LOWER': 'research'}]]

In [ ]:
[p.lemma_ for p in nlp("hypotheses")]

['hypothesis']

In [ ]:
singularize("hypotheses")

'hypothesis'

In [ ]:
plural_set = set()
for l in phr_patterns:
  if l[-1]["LOWER"] != singularize(l[-1]["LOWER"]):
    plural_set.add(l[-1]["LOWER"])

# [p[-1].pop("lower") for p in phr_patterns]
#  dictionary[new_key] = dictionary.pop(old_key)

In [ ]:
plural_list = list(plural_set)

In [ ]:
sorted(plural_list)

['algorithms',
 'altmetrics',
 'analyses',
 'analysis',
 'atlas.ti',
 'bayes',
 'bias',
 'caqdas',
 'cases',
 'categories',
 'cybermetrics',
 'data',
 'functions',
 'hermeneutics',
 'hierarchies',
 'hypotheses',
 'hypothesis',
 'informetrics',
 'intervals',
 'interviews',
 'jamovi',
 'machines',
 'maps',
 'means',
 'measures',
 'memos',
 'methodologies',
 'methods',
 'models',
 'networks',
 'optics',
 'packages',
 'perceptrons',
 'polygons',
 'process',
 'processes',
 'protocols',
 'scientometrics',
 'semantics',
 'smartpls',
 'squares',
 'statistics',
 'studies',
 'surveys',
 'synthesis',
 'tables',
 'tabulations',
 'topsis',
 'vignettes',
 'webometrics',
 'words']

In [ ]:
outs = ['analysis', 'atlas.ti', 'bayes', 'bias', 'caqdas','data','hypothesis', 'jamovi', 'means', 'process', 'smartpls','topsis']

In [ ]:
for l in phr_patterns:
  if (l[-1]["LOWER"] != singularize(l[-1]["LOWER"])) and (l[-1]["LOWER"] not in outs):
    l[-1]["LEMMA"] = singularize(l[-1]["LOWER"])
    l[-1].pop("LOWER")

In [ ]:
phr_patterns

[[{'LOWER': 'abduction'}],
 [{'LOWER': 'abduction'}, {'LOWER': 'reasoning'}],
 [{'LOWER': 'abductive'}],
 [{'LOWER': 'accidental'}, {'LOWER': 'sampling'}],
 [{'LOWER': 'action'}, {'LOWER': 'research'}],
 [{'LOWER': 'adaboost'}],
 [{'LOWER': 'adaptive'}, {'LOWER': 'boosting'}],
 [{'LOWER': 'adaptive'}, {'LOWER': 'conjoint'}, {'LOWER': 'analysis'}],
 [{'LOWER': 'adaptive'}, {'LOWER': 'sampling'}],
 [{'LOWER': 'affinity'}, {'LOWER': 'propagation'}],
 [{'LOWER': 'agglomerative'}, {'LOWER': 'clustering'}],
 [{'LEMMA': 'altmetric'}],
 [{'LOWER': 'analysis'}, {'LOWER': 'of'}, {'LOWER': 'covariance'}],
 [{'LOWER': 'analysis'}, {'LOWER': 'of'}, {'LOWER': 'variance'}],
 [{'LOWER': 'analytic'}, {'LOWER': 'hierarchy'}, {'LOWER': 'process'}],
 [{'LOWER': 'analytic'}, {'LEMMA': 'memo'}],
 [{'LOWER': 'analytical'}, {'LOWER': 'hiearachy'}, {'LOWER': 'process'}],
 [{'LOWER': 'analytical'}, {'LOWER': 'network'}, {'LOWER': 'process'}],
 [{'LOWER': 'analytical'}, {'LOWER': 'research'}],
 [{'LOWER': 'ancov

In [ ]:
phr_patterns_list = [{"label": "METHOD", "pattern": t} for t in phr_patterns]
phr_patterns_list[:25]

[{'label': 'METHOD', 'pattern': [{'LOWER': 'abduction'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'abduction'}, {'LOWER': 'reasoning'}]},
 {'label': 'METHOD', 'pattern': [{'LOWER': 'abductive'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'accidental'}, {'LOWER': 'sampling'}]},
 {'label': 'METHOD', 'pattern': [{'LOWER': 'action'}, {'LOWER': 'research'}]},
 {'label': 'METHOD', 'pattern': [{'LOWER': 'adaboost'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'adaptive'}, {'LOWER': 'boosting'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'adaptive'},
   {'LOWER': 'conjoint'},
   {'LOWER': 'analysis'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'adaptive'}, {'LOWER': 'sampling'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'affinity'}, {'LOWER': 'propagation'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'agglomerative'}, {'LOWER': 'clustering'}]},
 {'label': 'METHOD', 'pattern': [{'LEMMA': 'altmetric'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'analysis'},
   {

In [ ]:
acr = [t for t in tax_final_list if len(t)<6]
acr[:5]

['AHP', 'AMOS', 'ANN', 'ANOVA', 'ANP']

In [ ]:
acr_patterns_list = [{"label": "METHOD", "pattern": t} for t in acr]
acr_patterns_list[:5]

[{'label': 'METHOD', 'pattern': 'AHP'},
 {'label': 'METHOD', 'pattern': 'AMOS'},
 {'label': 'METHOD', 'pattern': 'ANN'},
 {'label': 'METHOD', 'pattern': 'ANOVA'},
 {'label': 'METHOD', 'pattern': 'ANP'}]

In [ ]:
pattern_list = phr_patterns_list + acr_patterns_list
pattern_list[:5]

[{'label': 'METHOD', 'pattern': [{'LOWER': 'abduction'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'abduction'}, {'LOWER': 'reasoning'}]},
 {'label': 'METHOD', 'pattern': [{'LOWER': 'abductive'}]},
 {'label': 'METHOD',
  'pattern': [{'LOWER': 'accidental'}, {'LOWER': 'sampling'}]},
 {'label': 'METHOD', 'pattern': [{'LOWER': 'action'}, {'LOWER': 'research'}]}]

In [ ]:
#with open("/content/drive/My Drive/Data/method_patterns_5Feb.jsonl", 'w') as f:
    for item in pattern_list:
        f.write(json.dumps(item) + "\n")

In [ ]:
# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab, validate=True)

In [ ]:
phr_patterns[:5]

[[{'LOWER': 'abduction'}],
 [{'LOWER': 'abduction'}, {'LOWER': 'reasoning'}],
 [{'LOWER': 'abductive'}],
 [{'LOWER': 'accidental'}, {'LOWER': 'sampling'}],
 [{'LOWER': 'action'}, {'LOWER': 'research'}]]

In [ ]:
pattern = phr_patterns + [[{"TEXT": t}] for t in acr]

In [ ]:
matcher.add("Hello", pattern)

In [ ]:
doc  = nlp ("abductive action research abduction")

In [ ]:
# Call the matcher on the doc
matches = matcher(doc)
matches

[(15777305708150031551, 0, 1),
 (15777305708150031551, 1, 3),
 (15777305708150031551, 3, 4)]

In [ ]:
matched_entities = []
for match_id, start, end in matches:
  matched_entities.append(doc[start:end].text)
print(matched_entities)

['abductive', 'action research', 'abduction']


In [ ]:
with open("/content/drive/My Drive/Data/sentences.pkl", "rb") as fp:   # Unpickling
  sents = pickle.load(fp)

In [ ]:
matched_entities = []
for s in sents:
  doc  = nlp (s)
  matches = matcher(doc)
  for match_id, start, end in matches:
    matched_entities.append(doc[start:end].text)

In [ ]:
matched_entities

['case study',
 'qualitative',
 'qualitative research',
 'triangulation',
 'case study',
 'Qualitative',
 'content analysis',
 'qualitative',
 'HE',
 'HE',
 'HE',
 'simple random sampling',
 'random sampling',
 'sampling',
 'SPSS',
 'online survey',
 'survey',
 'population',
 'systematic review',
 'literature review',
 'qualitative',
 'research method',
 'qualitative',
 'mixed method',
 'questionnaire',
 'panel data',
 'linear models',
 'forecasting',
 'forecasting',
 'survey',
 'Delphi method',
 'average',
 'network',
 'questionnaire',
 'survey',
 'survey research',
 'case study',
 'quantitative',
 'qualitative',
 'text mining',
 'qualitative',
 'discourse analysis',
 'empirical analysis',
 'validity',
 'hypothesis',
 'quantitative',
 'questionnaire',
 'descriptive statistics',
 'hypothesis',
 'online survey',
 'survey',
 'regression',
 'qualitative',
 'thematic analysis',
 'quantitative',
 'quantitative research',
 'questionnaire',
 'questionnaire survey',
 'survey',
 'questionnaire'

In [ ]:
len(matched_entities)

65590

In [ ]:
unique_matched_entities = list(set(matched_entities))
len(unique_matched_entities)

1044

In [ ]:
#with open("/content/drive/My Drive/Data/unique_matched_entities.pkl", "wb") as fp:   # Unpickling
#  pickle.dump(unique_matched_entities, fp)

In [ ]:
with open("/content/drive/My Drive/Data/unique_matched_entities.pkl", "rb") as fp:   # Unpickling
  ume = pickle.load(fp)

In [ ]:
len(ume)

1044

In [ ]:
not_found_entities = [t for t in tax_final_list if t not in ume]
len(not_found_entities)

370

In [ ]:
#with open("/content/drive/My Drive/Data/not_found_entities.pkl", "wb") as fp:   # Unpickling
#  pickle.dump(not_found_entities, fp)

In [ ]:
with open("/content/drive/My Drive/Data/not_found_entities.pkl", "rb") as fp:   # Unpickling
  nfe = pickle.load(fp)

In [ ]:
len(nfe)

370

In [ ]:
pd.DataFrame(tax_final_list).to_excel("/content/drive/My Drive/Data/taxfinallist_5Feb.xlsx")

In [ ]:
pd.DataFrame(nfe).to_excel("/content/drive/My Drive/Data/notfoundentities_5Feb.xlsx")